# 0. 라이브러리 임포트

In [99]:
#%pip install selenium
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import numpy
import warnings
import re
import requests
import urllib
import time
import selenium
from selenium.webdriver.common.by import By
from selenium import webdriver
import urllib
from urllib.request import urlopen 

options = webdriver.ChromeOptions()
# options.add_argument('headless') # 옵션 적용 시 에러 발생
driver = webdriver.Chrome(options = options)
warnings.filterwarnings(action='ignore')


# 1. 크롤링
개요, 연혁, 교육이념, 교육목표, 전공소개, 학과 안내, 강의교과목

## 1-1. 중앙대학교

In [100]:
# 세부 페이지 접속 함수
def go_to(dept1_idx, dept2_idx):
    driver = webdriver.Chrome(options = options)
    driver.get("https://lis.cau.ac.kr/")
    time.sleep(1)
    driver.find_element('xpath','//*[@id="gnavigation"]').click()
    driver.find_element('xpath','//*[@id="gnavigation"]/ul/li[1]/a').click()
    driver.find_element('xpath','//*[@id="gnavigation"]/ul/li[1]/div/div/ul/li[%s]/a[%s]'%(dept1_idx, dept2_idx)).click()
    return driver

In [101]:
# 크롤링
cau = ' '

# 개요 / 연혁
driver = go_to(1,1)
cont1 = driver.find_element('xpath','//*[@id="contents"]/dl[1]/dd[4]').text
cont2 = driver.find_element('xpath','//*[@id="contents"]/dl[1]/dd[6]').text
cau = cau + cont1+cont2
driver.close()

# 교육 이념
driver = go_to(1,2)
cont3 = driver.find_element('xpath', '//*[@id="contents"]/dl[1]/dd[3]/span').text
cont4 = driver.find_element('xpath','//*[@id="contents"]/dl[1]/dd[6]/span').text
cont5 = driver.find_element('xpath','//*[@id="contents"]/dl[1]/dd[8]/span').text
cont6 = driver.find_element('xpath','//*[@id="contents"]/dl[1]/dd[10]/span').text
cont7 = driver.find_element('xpath','//*[@id="contents"]/dl[1]/dd[12]').text
cont8 = driver.find_element('xpath','//*[@id="contents"]/dl[1]/dd[14]/span').text
cau = cont3+cont4+cont5+cont6+cont7+cont8+cau
driver.close()

# 교과 과정
temp_li=[]
driver = go_to(3,3)

td = driver.find_elements('xpath','//*[@id="contents"]/dl[1]/table[1]/tbody')
td = td[0].text.split('\n')
temp_li += td
for x in range(1,5,1):
    text= driver.find_element('xpath','//*[@id="contents"]/dl[1]/div[%s]'%str(x)).text
    temp_li.append(text)
for x in temp_li:
    cau += x
driver.close()

## 1-2. 연세대학교

In [127]:
# 세부 페이지 접속 함수
def go_to_y(dept1_idx, dept2_idx):
    driver = webdriver.Chrome(options = options)
    driver.get("https://yslis.yonsei.ac.kr/doai/index.do")
    time.sleep(1)
    
    # 마우스 오버 
    action_dir = driver.find_element('xpath','//*[@id="item_body"]/header/div[2]/div/div/ul/li[%s]/a'%dept1_idx)
    act = webdriver.ActionChains(driver)
    act.move_to_element(action_dir).perform() # 마우스 오버 시켜야 상세 메뉴 나타남
    act.perform()

    # 요소 찾기
    driver.find_element('xpath','//*[@id="item_body"]/header/div[2]/div/div/ul/li[%s]/ul/li[%s]/a'%(dept1_idx, dept2_idx)).click()
    return driver

In [125]:
# 크롤링
yon = ''

# 학과 소개
driver  = go_to_y(1,1)
t = driver.find_element('xpath','//*[@id="jwxe_main_content"]/div').text
yon += t
driver.close()

# 학과 연혁
driver = go_to_y(1,2)
t = driver.find_element('xpath','//*[@id="his2000"]').text
yon += t
driver.find_element('xpath','//*[@id="jwxe_main_content"]/div/div/div[1]/ul/li[2]/a').click()
t2 = driver.find_element('xpath','//*[@id="his1990"]').text
yon += t2
driver.close()

# 교과 과정
driver = go_to_y(3,1)
t = driver.find_element('xpath','//*[@id="jwxe_main_content"]').text
yon += t
driver.close()


## 1-3. 성균관대학교

In [130]:
# 세부 페이지 접속 함수
def go_to_s(dept1_idx, dept2_idx):
    driver = webdriver.Chrome(options = options)
    driver.get("https://lis.skku.edu/lis/index.do")
    driver.maximize_window()
    time.sleep(1)
    
    # 마우스 오버 
    action_dir = driver.find_element('xpath','//*[@id="item_body"]/header/div[2]/div[2]/div/div[2]/div/ul/li[%s]/a'%dept1_idx)
    act = webdriver.ActionChains(driver)
    act.move_to_element(action_dir).perform() # 마우스 오버 시켜야 상세 메뉴 나타남
    act.perform()

    # 요소 찾기
    driver.find_element('xpath','//*[@id="item_body"]/header/div[2]/div[2]/div/div[2]/div/ul/li[%s]/ul/li[%s]/a'%(dept1_idx, dept2_idx)).click()
    return driver


In [145]:
# 크롤링
skk = ' '

# 인사말  - 준비중

# 학과 연혁
driver = go_to_s(1,2)
t = driver.find_element('xpath', '//*[@id="jwxe_main_content"]').text
skk += t
driver.close()

# 교육 목표 및 학과 안내
driver = go_to_s(1,3)
t = driver.find_element('xpath','//*[@id="jwxe_main_content"]').text
skk += t
driver.close()

# 교육과정
driver = go_to_s(2,1)
for i in range(2,5,1):
    t = driver.find_element('xpath','//*[@id="jwxe_main_content"]/div[1]/div/table/tbody').text
    skk += t
    driver.find_element('xpath','//*[@id="jwxe_main_content"]/div[2]/ul/li[%s]'%i).click()
# 과목코드, 개설여부 -> 전처리 단계에서 다른 학교들과 한번에 처리하려면 복잡해짐 -> 미리 전처리
skk = skk.replace('Yes','')
skk = skk.replace('No','')
re_comp  = re.compile('[A-Z]{3}[0-9]{3}') # 과목코드 제거
skk = re_comp.sub('',skk)
skk = re.sub('[0-9]*[0-9]','',skk) # 이수 학기 제거
skk = re.sub("[0-9]*' '*[0-9]",'',skk) # 학점 제거
driver.close()
# 과정소개  - 준비중
